                    GNU GENERAL PUBLIC LICENSE
                       Version 3, 29 June 2007

In [ ]:
from Bio import SeqIO
# para leer fasta

In [ ]:
#lectura de secuencias objetivo
records=list(SeqIO.parse("DQB 9F.fa", "fasta"))

#Aqui deberia prever la lectura de los leidos del cromatograma en R

In [ ]:
records[0]

In [ ]:
#convierto las secuencias a strings y dscripciones
seq=[str(r.seq) for r in records]
desc=[str(r.description).strip() for r in records]
print desc[1]
print seq[1]
#convierto las secuencias en strings - OJO QUE DEJO EL ID


In [ ]:
def seq_a_lista(s):
    #recibe un string y devuelve una lista
    # proceso las [A/G] eliminando los [] y uniendo los dos valores en torno a / como listas
    # TODO: ver que pasa con los [./.]. De movida debe quedar .. ya que no estoy usando sets, que eliminarian la duplicación
    s=s.replace('[','').replace(']','')
    sl= [x for x in s]
    for i,x in enumerate(sl):
        if x=='/':
            sl[i-1]+=sl[i+1]
            del sl[i:i+2]
    return sl

def lista_a_seq(l):
    '''recibe una lista y devuelve un string con corchetes y barra en los polimorfismos
    '''
    ll=[x if len(x)<2 else '[' + x[0] + '/' + x[1] + ']' for x in l] #'/'.join(x)
    return ''.join(ll)
    
def seq_a_string(s):
    '''recibe un string con corchetes, lo transforma en lista y lo traduce a IUPAC
    '''
    return lista_a_iupac(seq_a_lista(s))

def guion_a_vacio(l): # creo que ya no se usa, ver. recibe un string o una lista y convierte guiones en vacíos
    return [ x if (x<>'-')  else '' for x in la]

def vacio_a_guion(l):
    '''recibe una lista y convierte los vacíos en guiones -
       si recibe un string lo devuelve tal cual
    '''
    return [ x if x else '-' for x in l]

def unir_haplotipos(la,lb):
    '''recibe dos listas de haplotipos de referencia
       maneja secuencias con guiones y las convierte a listas con vacíos
       devuelve una lista de la union de las dos listas elemento a elemento como conjunto (sin duplicados)
    '''
    la=[ x if (x<>'-')  else '' for x in la]
    lb=[ x if (x<>'-')  else '' for x in lb]
    return [''.join(list(set(la[i]+lb[i]))) for i in range(len(la))]

def ref_haplo(la):
    '''recibe un haplotipo de referencia, con los guiones convertidos en vacíos
       completa los guiones - con los datos del haplotipo de referencia
       
    '''
    return [ refseq[i] if not x or x=='-'  else x for i,x in enumerate(la)]
    #return [ x if (x in 'ACGT')  else refseq[i] for i,x in enumerate(la)] ## no son equivalentes, y este funciona siempre? Ver   

In [ ]:
print len(seq)

In [ ]:
for s in seq: #imprimo separado por espacios para ver
    print ' '.join(seq_a_lista(s))

In [ ]:
#lectura del alineamiento
f=open('alineamiento DQB1.txt')
lineas=f.readlines()

# Queda en el diccionario d hasheado por id
d={}
ref=''
for i,l in enumerate(lineas):
    if (len(l)>20 and l[0]==' ' and l[1]!=' '): # uso un patron para identificar las lineas con datos
        id= l[:20].strip() #las corto por ancho fijo del id
        #print i, id
        sseq= l[21:].replace(' ','').rstrip() # elimino espacis
        #print id
        #print seq
        if id in d.keys(): # y las guardo (concatenadas) en un diccionario indexado por id
            d[id]+=sseq
        else:
            d[id]=sseq
        if not ref: ref=id # me estoy quedando con la primera secuencia asumiendo que es la de referencia
        d[id]=d[id].replace('*','-')
f.close()


In [ ]:
# no hace falta, pero pongo la secuencia de referencia en esta variable (despues reorganizo) ## La uso??
ref
refseq = d[ref]
print ref, refseq

In [ ]:
seqs=[]
for s in seq:
    seqs.append(seq_a_lista(s))
print seqs

In [ ]:
print seqs[0]

In [ ]:
print len(d), d.keys()

In [ ]:
#Generación de todos los haplotipos posibles
haplos=[]
hh=d.keys()
f=open('haplos.log','w')

def log(l): #aca defino la funcion para generar los logs
    for x in l:
        f.write(str(x))
    f.write('\n')
    
for x in hh:
    for y in hh:
        log((x,y))#print x,y
        log(('x :', d[x]))#print 'x:', d[x]
        log(('y :', d[y]))#print 'y:', d[y]
        log(('xr:',ref_haplo(d[x])))
        log(('yr:',ref_haplo(d[y])))
        dd= unir_haplotipos(ref_haplo(d[x]), ref_haplo(d[y]))
        log(('z0:', lista_a_seq((dd))))
        log(('z1:', lista_a_seq(vacio_a_guion(dd))))
        log(('z2:', lista_a_seq(dd)))
        log(('z3:', dd))
        haplos.append([[x,y],dd])

f.close()

- Leer archivo(s) de lecturas
- Leer archivo(s) de haplotipos
- Generar archivo de ranking
    En el original:
        * lectura, par de haplos, ranking, diferencias


In [ ]:
print len(haplos), len(hh), len(hh)*len(hh)
print hh[11], haplos[11]

In [ ]:
print len(seqs)

In [ ]:
for ss in seqs:
    print lista_a_seq(ss)

In [ ]:
s=seqs[0]
print s
print ':', lista_a_seq(s)
#print s, len(s)
print 'haplos:', haplos[4]
h=ref_haplo(haplos[4][1])
print 'h:',lista_a_seq(h)
#print h, len(h)

In [ ]:
print refseq
print h


In [ ]:
IUPAC={
'A':'A',
'C':'C',
'G':'G',
'T':'T',
'U':'U',
'AG':'R',
'CT':'Y',
'CG':'S',
'AT':'W',
'GT':'K',
'AC':'M',
'CGT':'B',
'AGT':'D',
'ACT':'H',
'ACG':'V',
'ACGT':'N',
'.':'.',
'-':'-'
}

In [ ]:
IUPAC['AG']

In [ ]:
IUPAC_1=dict([ [x[1],x[0]] for x in list(IUPAC.items())])

In [ ]:
IUPAC_1['H']

In [ ]:
s='ACGTRYSWMBHVN'
s='AHRV'
def iupac_a_seq(s):
    t=[]
    for x in s:
        t.append(IUPAC_1[x])
    return t
iupac_a_seq(s)

In [ ]:
s=['A', 'C', 'G', 'T', 'GA', 'CT', 'CG', 'AT', 'AC', 'CGT', 'ACT', 'ACG', 'ACGT']

def lista_a_iupac(s):
    ''' Recibe una lista del tipo del ejemplo, la pasa a iupac, devuelve string
    '''
    t=[]
    for x in s:
        t.append(IUPAC[''.join(sorted(list(x)))])
    return ''.join(t)
lista_a_iupac(s)

In [ ]:
def seq_compare(s,t): #se supone que estan alineados y tienen la misma longitud
    match=0 
    nomatch=0 
    compared=0
    nocompared=0
    total=len(s)
    #print 'S T i M N C'
    for i, x in enumerate(s):
        y=t[i]
        if x=='-' or y=='-':
            nocompared+=1
        elif x==y: 
            match +=1
            compared +=1
        else:
            nomatch +=1
            compared +=1
        #print x, t[i], i, match, nomatch, compared
        
    return (match, nomatch, compared, nocompared, total)

def haplo_compare(s,t): #se supone que estan alineados y tienen la misma longitud
    comp=[]
    match=0 
    nomatch=0 
    compared=0
    nocompared=0
    total=len(s)
    #print 'S T i M N C'
    for i, x in enumerate(s):
        yi=set(t[i])
        xi=set(x)
        if xi==set('-') or yi==set('-'):
            nocompared+=1
            comp +='-'
        elif xi.issubset(yi): 
            match +=1
            compared +=1
            comp +='|'
        else:
            nomatch +=1
            compared +=1
            comp +='X'
        #print x, t[i], i, match, nomatch, compared
        
    return [match, nomatch, compared, nocompared, total, ''.join(comp)]
seq_compare('ASDFGH','ASEF-H')

In [ ]:
seq_compare(refseq, refseq)

In [ ]:
# prueba del algoritmo de alineamiento
# para adaptarse a las diferencias sin introducir gap
# -> local y no global, penalizando altamente el gap (habria que buscar un mínimo)
f=open('alineamiento.txt','w')
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
s=lista_a_seq(s)
h1=lista_a_seq(h[1])
alignments = pairwise2.align.localms(refseq,h1[:200],2,-1,-5,-10)
log(('#alignements=',len(alignments)))
for a in alignments:
    log(('--------------'))
    log((format_alignment(*a)))
    log(('a=',a))
    log(('seq_compare',seq_compare(a[0], a[1])))
#    print '--------------'
#    print format_alignment(*a)
#    print a
#    print seq_compare(a[0], a[1])

def alinear(s,t):
#    print 'align:'
#    print 's:', s
#    print 't:', t
    #log(('align:'))
    #log(('s:', s))
    #log(('t:', t))
    al = pairwise2.align.localms(s,t,2,-1,-5,-10)
#    for a in al:
 #       log(('--------------'))
  #      log((format_alignment(*a)))
   #     log(('a=',a))
    #    log(('seq_compare',seq_compare(a[0], a[1])))
    
    #log((a[0][0], a[0][1]))
    #log(('pairwise'))
    #log((a))

    return al[0][0], al[0][1]
f.close()

In [ ]:
s

In [ ]:
print h[1]

In [ ]:
q='GTCATYTGDGTDGCDGGYTCDCC'
q='TTGAGBARRCCRTACATRAGC'
q='GGGCTRACRAARGCYCKRTCCC'
q='CCHTACAAYGCYGATTTYGACG'
q='GCTYATGTAYGGYYTVCTCAA'
q='GCTYTDCCKGGYATGAAYGTYCG'
r=iupac_a_seq(q)
print r

In [ ]:
#esto es lo que hice para Suani!!!

def gen(s,a=''):
    if len(s)>1:        #print len(s)
        for x in s[0]:
            gen(s[1:],a+x)
    else:               #es el ultimo, cerrar
        for x in s[0]:
            print a + x
s=r  
gen(s) 

In [ ]:
hap=open('haplos.txt','w')
for h in haplos:
    hap.write(str(h[0]) + ' ' + lista_a_seq(h[1]) + '\n')
hap.close()

bak='''
for i,s in enumerate(seq):
    f=open(desc[i]+'comparacion.txt','w')
    print i,':',s
    for j,h in enumerate(haplos):
        print j,
        h1=h[1]
        s1=seq_a_lista(s)
        h1=h[1]
        log(('------------------\n')) #print '-------------------'
        x, y= alinear(lista_a_iupac(s1),lista_a_iupac(h1)) #
        res=haplo_compare(x,y)
        log((h[0]))
        log((x))
        log((str(''.join(res[-1]))))
        log((y))
        log(('match, nomatch, compared, nocompared, total, comp'))
        log((str(res[:-1])+'\n'))
    f.close()
'''

In [ ]:
for i,s in enumerate(seq):
    r=open(desc[i]+'comparacion.txt','w')
    rank=[]
    print i,':',s
    for j,h in enumerate(haplos):
        print j,
        h1=h[1]
        #print 'h1:',h1
        #print lista_a_iupac(h1)
        s1=seq_a_lista(s)
        #print 's1:',s1
        #print lista_a_iupac(s1)
        #log(('------------------\n')) #print '-------------------'
        x, y= alinear(lista_a_iupac(s1),lista_a_iupac(h1)) #
        res=haplo_compare(x,y)
        res.extend([h[0],x,y])
        rank.append(res)
        debug='''
        log((h[0])) # los id
        log((x))
        log((str(''.join(res[-1]))))
        log((y))
        log(('match, nomatch, compared, nocompared, total, comp'))
        log((str(res[:-1])+'\n'))
        print((h[0])) # los id
        print((x))
        print((str(''.join(res[-1]))))
        print((y))
        print(('match, nomatch, compared, nocompared, total, comp'))
        print((str(res[:-1])+'\n'))
        '''
    #print 'sorting...'
    rank.sort(reverse=True)
    #r=open('rank.txt','w')
    for x in rank:
        r.write('Haplotipo: '+ ' '.join(x[6])+'\n')
        r.write(', '.join([str(xx) for xx in x[:5]])+'\n')
        r.write(str(x[7])+'\n')
        r.write(str(x[5])+'\n')
        r.write(str(x[8])+'\n\n')
    
    r.close()

In [ ]:
rank.sort(reverse=True)
r=open('rank.txt','w')
for x in rank:
    r.write('Haplotipo: '+ ' '.join(x[6])+'\n')
    r.write(', '.join([str(xx) for xx in x[:5]])+'\n')
    r.write(str(x[7])+'\n')
    r.write(str(x[5])+'\n')
    r.write(str(x[8])+'\n\n')
    
r.close()

In [ ]:
 f.close()

In [ ]:
s=seq_a_lista(seq[2])
h=haplos[22]
t=h[1]
print 'S:', s
print ' '.join(s)
print 'S_iupac:'
si= ''.join(lista_a_iupac(s))
print 'si:', si
print 'haplo:',h[0]
print t
ti=seq_a_iupac(t)
print ti
ti =''.join(ti)
print 'ti:',ti

In [ ]:
alignments = pairwise2.align.localms(si,ti,2,-1,-5,-10)

In [ ]:
for a in alignments:
    print '--------------'
    print format_alignment(*a)
    print a
    print seq_compare(a[0], a[1])

In [ ]:
print a[0]
print a[1]
print seq_compare(a[0],a[1])

In [ ]:
print iupac_a_seq(a[0])
print iupac_a_seq(a[1])
print haplo_compare(iupac_a_seq(a[0]),iupac_a_seq(a[1]))



In [ ]:
print haplo_compare(iupac_a_seq(a[1]),iupac_a_seq(a[0]))


In [ ]:
p= iupac_a_seq(a[0])
q= iupac_a_seq(a[1])
r= haplo_compare(iupac_a_seq(a[0]),iupac_a_seq(a[1]))
print ''.join(p)
print ''.join(q)
print ''.join(r[-1])

In [ ]:
f=open('prisec.txt')
ps=f.readlines()
f.close()
print len(ps)
a=ps[0]
b=ps[1]
print a
print b

In [ ]:
ab=unir_haplotipos(a,b)

In [ ]:
abseq=lista_a_seq(ab)
print abseq

In [ ]:
seq_a_lista(abseq)

In [ ]:
seq[0]

In [ ]:
seqs[0]